In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime
import time
import os
import pandas
import requests
import boto3
import numpy as np
import xarray as xr
import rasterio as rio
from rasterio.session import AWSSession
from rasterio.plot import show
import rioxarray

In [ ]:
# get credentials
s3_cred_endpoint = 'https://data.lpdaac.earthdatacloud.nasa.gov/s3credentials'
def get_temp_creds():
    temp_creds_url = s3_cred_endpoint
    return requests.get(temp_creds_url).json()

temp_creds_req = get_temp_creds()

session = boto3.Session(aws_access_key_id=temp_creds_req['accessKeyId'], 
                        aws_secret_access_key=temp_creds_req['secretAccessKey'],
                        aws_session_token=temp_creds_req['sessionToken'],
                        region_name='us-west-2')

In [ ]:
session

In [ ]:
rio_env = rio.Env(AWSSession(session),
                  GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR',
                  GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                  GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'))
rio_env.__enter__()

In [ ]:
# read the .csv file with S3 links
stack_df = pandas.read_csv('/home/jovyan/nch21_hls_timeseries/HLS_data/T13TDE/stack.csv')
stack_df = stack_df.loc[~stack_df['date'].isna(), :]
stack_df.reset_index(inplace=True)
stack_df

In [ ]:
# subset the s3 links by band
header = ['label', 'L30_band', 'S30_band', 'read']
data = [
    ['coastal_aerosol', 'B01', 'B01', False],
    ['blue', 'B02', 'B02', True],
    ['green', 'B03', 'B03', True],
    ['red', 'B04', 'B04', True],
    ['red-edge_1', None, 'B05', False],
    ['red-edge_2', None, 'B06', False],
    ['red-edge_3', None, 'B07', False],
    ['nir_broad', None, 'B08', False],
    ['nir', 'B05', 'B8A', True],
    ['swir_1', 'B06', 'B11', True],
    ['swir_2', 'B07', 'B12', True],
    ['water_vapor', None, 'B09', False],
    ['cirrus', 'B09', 'B10', False],
    ['thermal_infrared_1', 'B10', None, False],
    ['thermal_infrared_2', 'B11', None, False],
    ['fmask', 'Fmask', 'Fmask', True]
]

band_df = pandas.DataFrame(data, columns=header)
band_df

In [ ]:
%%time

chunks=dict(band=1, x=256, y=256)

hls_ds = None

for i in range(0, band_df.shape[0]):
    if band_df.loc[i, 'read'] == True:
        # subset stack for links for each band
        band_stack = stack_df.loc[
        ((stack_df['band'] == band_df.loc[i,'L30_band']) & (stack_df['sensor'] == 'L30')) |
        ((stack_df['band'] == band_df.loc[i,'S30_band']) & (stack_df['sensor'] == 'S30')), :]
        
        # create the time index
        band_time = [datetime.strptime(str(t), '%Y%jT%H%M%S') for t in band_stack['date']]
        xr.Variable('time', band_time)

        #s3_links = band_stack['S3_links']
        s3_links = band_stack['local_links']
        
        # get the band label
        band_label = band_df.loc[i, 'label']
        
        # open the links
        hls_ts_da = xr.concat([rioxarray.open_rasterio(f, chunks=chunks).squeeze('band', drop=True) for f in s3_links], dim=band_time)
        hls_ts_da = hls_ts_da.rename({'concat_dim':'time'})
        
        if hls_ds is None:
            hls_ds = xr.Dataset({band_label: hls_ts_da})
        else:
            hls_ds[band_label] = hls_ts_da

#hls_ds.rename({'concat_dim':'time'})
hls_ds

In [ ]:
%%time

def SI(b1, b2):
    si = (b1 - b2) / (b1 + b2)
    si = xr.where(si < -1.0, -1.0, si)
    si = xr.where(si > 1.0, 1.0, si)
    si = xr.where(np.isfinite(si), si, np.nan)
    return(si)

# calculate NDVI (normalized difference vegetation index)
hls_ds['ndvi'] = SI(hls_ds['nir'], hls_ds['red'])

# calculate NBR (normalized difference burn ratio)
hls_ds['nbr'] = SI(hls_ds['nir'], hls_ds['swir_2'])

# calculate NDWI (normalized difference water index)
hls_ds['ndwi_gao'] = SI(hls_ds['nir'], hls_ds['swir_1'])
hls_ds['ndwi_mcfeeters'] = SI(hls_ds['green'], hls_ds['nir'])

# calculate NDSI (normalized difference snow index)
hls_ds['ndsi'] = SI(hls_ds['green'], hls_ds['swir_1'])

hls_ds

In [ ]:
%%time

hls_ds['fmask_cirrus'] = (hls_ds['fmask'] & 0b1) > 0
hls_ds['fmask_cloud'] = (hls_ds['fmask'] & 0b10) > 0
hls_ds['fmask_adjacent'] = (hls_ds['fmask'] & 0b100) > 0
hls_ds['fmask_shadow'] = (hls_ds['fmask'] & 0b1000) > 0
hls_ds['fmask_snow'] = (hls_ds['fmask'] & 0b10000) > 0
hls_ds['fmask_water'] = (hls_ds['fmask'] & 0b100000) > 0
hls_ds['fmask_aerosol6'] = (hls_ds['fmask'] & 0b1000000) > 0
hls_ds['fmask_aerosol7'] = (hls_ds['fmask'] & 0b10000000) > 0

In [ ]:
import geoviews as gv
import hvplot.xarray
import holoviews as hv
gv.extension('bokeh', 'matplotlib')

In [ ]:
# check fmask masks for a given scene
# ignore water mask?
# ignore aerosol levels

i = 21 # which scene in the dataseries to examine
print(hls_ds['time'][i])

tt = hls_ds[['fmask_cirrus', 'fmask_cloud', 'fmask_adjacent', 'fmask_shadow', 'fmask_snow', 'fmask_water', 'fmask_aerosol6', 'fmask_aerosol7']]
tt['fmask_cirrus'][i, :, :].hvplot(x='x', y='y', cmap='viridis', title='Cirrus clouds') + \
tt['fmask_cloud'][i, :, :].hvplot(x='x', y='y', cmap='viridis', title='Clouds')

In [ ]:
tt['fmask_shadow'][i, :, :].hvplot(x='x', y='y', cmap='viridis', title='Cloud shadows') + \
tt['fmask_adjacent'][i, :, :].hvplot(x='x', y='y', cmap='viridis', title='Adjacent to cloud shadows')

In [ ]:
tt['fmask_snow'][i, :, :].hvplot(x='x', y='y', cmap='viridis', title='Snow/ice') + \
tt['fmask_water'][i, :, :].hvplot(x='x', y='y', cmap='viridis', title='Water')

In [ ]:
tt['fmask_aerosol6'][i, :, :].hvplot(x='x', y='y', cmap='viridis', title='Aerosol level (bit 6)') + \
tt['fmask_aerosol7'][i, :, :].hvplot(x='x', y='y', cmap='viridis', title='Aerosol level (bit7)')

In [ ]:
# plot NBR for 1 image
hls_nbr = hls_ds['nbr'][i, :, :]
hls_ndwi = hls_ds['ndwi_mcfeeters'][i, :, :]
hls_ndsi = hls_ds['ndsi'][i, :, :]
hls_nbr.hvplot.image(x='x', y='y', rasterize=True, colorbar=True, cmap='viridis', title='NBR').opts(clim=(-0.5, 0.5)) + \
hls_ndwi.hvplot.image(x='x', y='y', rasterize=True, colorbar=True, cmap='viridis', title='NDWI').opts(clim=(-0.5, 0.5)) + \
hls_ndsi.hvplot.image(x='x', y='y', rasterize=True, colorbar=True, cmap='viridis', title='NDSI').opts(clim=(-0.5, 0.5))

# plot NBR for lots of images - this crashes the kernel because it uses too much memory
#hls_ds['nbr'].hvplot(groupby='time', width=1220, cmap='BrBG', widget_location='bottom').opts(clim=(-1, 1))

# plot qa values for 1 scene
# https://matplotlib.org/stable/gallery/color/named_colors.html
# clear (green), aerosol (250, light blue), water (251, blue), snow (252, cyan), clouds (253, white), cloud shadows (254, dark grey), nodata (255, black)
#color_list = ['forestgreen'] * 250 + ['powderblue', 'dodgerblue', 'cyan', 'white', 'darkgray', 'black']
#level_list = [0, 250, 251, 252, 253, 254, 255, 256]
#print(len(color_list))

#hls_ts_da_data = hls_ds['qa'][12, :, :]
#hls_ts_da_data.hvplot.image(x='x', y='y', rasterize=True, colorbar=True, cmap=color_list) #.opts(clim=(0, 250, 251, 252, 253, 254, 255))
#hls_ts_da_data.hvplot.image(x='x', y='y', rasterize=False, colorbar=True, color=color_list) #.opts(clim=(0, 250, 251, 252, 253, 254, 255))
#hls_ts_da_data.plot(levels=level_list, colors=color_list)


In [ ]:
##########
# stretch parameters
# min, max, mean, sd, n
##########
lndsr_scale = {
    'swir_2':[0.0, 4000.0, 2000.0, 1000.0, 2.0], \
    'nir':[0.0, 4000.0, 2000.0, 1000.0, 2.0], \
    'blue':[0.0, 2000.0, 1000.0, 500.0, 2.0] \
}

##########
# apply sd stretch to a band
##########
def sd_stretch(x, params, nodata=-9999, q=255):
    min_clip = params[0]
    max_clip = params[1]
    mean = params[2]
    sd = params[3]
    n = params[4]

    min_stretch = mean - (n * sd)
    max_stretch = mean + (n * sd)

    x[x == nodata] = 0
    x[x <= min_clip] = 0
    x[x >= max_clip] = max_stretch

    x_stretch = q * (x - min_stretch) / (max_stretch - min_stretch)

    return(x_stretch.astype(np.uint8))

In [ ]:
%%time

red = hls_ds['swir_2'][i, :, :]
green = hls_ds['nir'][i, :, :]
blue = hls_ds['blue'][i, :, :]

r = sd_stretch(red.values, lndsr_scale['swir_2'])
g = sd_stretch(green.values, lndsr_scale['nir'])
b = sd_stretch(blue.values, lndsr_scale['blue'])

rgb = hv.RGB(np.dstack([r/255.0, g/255.0, b/255.0, np.ones(b.shape)]))

In [ ]:
%%time
rgb
#hv.Image(rgb)